In [26]:
%pip install geopandas
%pip install geopy
%pip install gmaps
%pip install pyarrow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     |████████████████████████████████| 35.3 MB 4.8 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import json

cfg = json.load(open('secure/configuration.json'))

df = pd.read_csv('schulsuche_export.csv', delimiter=';', encoding='ISO-8859-1')

In [7]:
cfg

{'google_maps_api_key': 'abcdef'}

In [12]:
df.Schultyp.unique()

array(['Gymnasium', 'Realschule',
       'Volksschule zur sonderp?dagog. F?rderung',
       'Sonstige Berufsfachschule',
       'Berufsfachschule des Gesundheitswesens', 'Gewerbliche Fachschule',
       'Berufsfachschule f. Fremdsprachenberufe',
       'Kaufm?nnische Berufsfachschule', 'Grund- u. Mittelschule',
       'Fachoberschule', 'Berufsoberschule', 'Wirtschaftsschule',
       'Hauswirtschaftliche Fachschule', 'Berufsschule',
       'Berufsfachs. f. Hauswirtschaft u. Sozialberufe', 'Fachakademie',
       'Berufsschule zur sonderp?dagog. F?rderung',
       'Gewerbliche Berufsfachschule',
       'Berufsfachschule f. techn. Assistenzberufe', 'Abendrealschule',
       nan, 'Freie Waldorfschule',
       'Realschule zur sonderp?dagog. F?rderung', 'Ausl?ndische Schule',
       'Schule besonderer Art', 'Berufsfachschule f. Musik',
       'Abendgymnasium', 'Kolleg', 'Sonstige Fachschule',
       'Landwirtschaftliche Fachschule',
       'Fachakademie f?r Landwirtschaft', 'Kaufm?nnische Fac

In [3]:
schools_of_interest = pd.concat([df[df.Schultyp=='Gymnasium'], df[df.Schultyp=='Freie Waldorfschule']], ignore_index=True)

In [23]:
schools_of_interest

,Schulnummer,Schultyp,Name,Strasse,PLZ,Ort,Homepage,Link
0,13,Gymnasium,Kronberg-Gymnasium Aschaffenburg,Fasaneriestr. 33,63739,Aschaffenburg,kronberg-gymnasium.de,/schule/0013.html
1,15,Gymnasium,Karl-Theodor-von-Dalberg-Gymnasium Aschaffenbu...,Grünewaldstr. 18,63739,Aschaffenburg,www.dalberg-gymnasium.de/,/schule/0015.html
2,16,Gymnasium,Maria-Ward-Schule Mädchengymnasium der Maria-W...,Brentanoplatz 8,63739,Aschaffenburg,mws-ab.de,/schule/0016.html
3,14,Gymnasium,Friedrich-Dessauer-Gymnasium Aschaffenburg,Stadtbadstr. 4,63741,Aschaffenburg,https://www.fdg-online.de,/schule/0014.html
4,3,Gymnasium,Spessart-Gymnasium Alzenau,Brentanostr. 55,63755,Alzenau,www.spessart-gymnasium.de,/schule/0003.html
...,...,...,...,...,...,...,...,...
454,1041,Freie Waldorfschule,Priv. Waldorfschule Hof,Kolpingshöhe 3,95032,Hof,www.waldorfschule-hof.de,/schule/1041.html
455,1022,Freie Waldorfschule,Freie Waldorfschule Wernstein des Junger Main ...,Am Eisweiher 1,95336,Mainleus,www.schule-wernstein.de,/schule/1022.html
456,1037,Freie Waldorfschule,Rudolf-Steiner-Schule Coburg,Callenberg 12,96450,Coburg,www.waldorfschule-coburg.de,/schule/1037.html
457,1018,Freie Waldorfschule,Freie Waldorfschule Würzburg Schule nach der P...,Oberer Neubergweg 14,97074,Würzburg,www.waldorfschule-wuerzburg.de,/schule/1018.html


Add the GPS coordinates for all schools of type "Gymnasium".

In [4]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

geolocator = Nominatim(user_agent="address_to_gps_converter")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

def address_to_gps(strasse, plz, ort):
    address_joined = ' '.join([strasse, str(plz), ort])
    location = geolocator.geocode(address_joined)
    return (location.latitude, location.longitude) if location is not None else (None, None)

schools_of_interest[['latitude', 'longitude']] = pd.DataFrame(schools_of_interest.apply(lambda row: address_to_gps(row.Strasse, row.PLZ, row.Ort), axis=1).tolist(), index=schools_of_interest.index)
schools_of_interest

,Schulnummer,Schultyp,Name,Strasse,PLZ,Ort,Homepage,Link,latitude,longitude
0,13,Gymnasium,Kronberg-Gymnasium Aschaffenburg,Fasaneriestr. 33,63739,Aschaffenburg,kronberg-gymnasium.de,/schule/0013.html,49.979046,9.162566
1,15,Gymnasium,Karl-Theodor-von-Dalberg-Gymnasium Aschaffenbu...,Grünewaldstr. 18,63739,Aschaffenburg,www.dalberg-gymnasium.de/,/schule/0015.html,49.974713,9.158185
2,16,Gymnasium,Maria-Ward-Schule Mädchengymnasium der Maria-W...,Brentanoplatz 8,63739,Aschaffenburg,mws-ab.de,/schule/0016.html,49.970401,9.153117
3,14,Gymnasium,Friedrich-Dessauer-Gymnasium Aschaffenburg,Stadtbadstr. 4,63741,Aschaffenburg,https://www.fdg-online.de,/schule/0014.html,49.973574,9.131572
4,3,Gymnasium,Spessart-Gymnasium Alzenau,Brentanostr. 55,63755,Alzenau,www.spessart-gymnasium.de,/schule/0003.html,50.092117,9.071371
...,...,...,...,...,...,...,...,...,...,...
454,1041,Freie Waldorfschule,Priv. Waldorfschule Hof,Kolpingshöhe 3,95032,Hof,www.waldorfschule-hof.de,/schule/1041.html,50.295222,11.917233
455,1022,Freie Waldorfschule,Freie Waldorfschule Wernstein des Junger Main ...,Am Eisweiher 1,95336,Mainleus,www.schule-wernstein.de,/schule/1022.html,50.120249,11.396585
456,1037,Freie Waldorfschule,Rudolf-Steiner-Schule Coburg,Callenberg 12,96450,Coburg,www.waldorfschule-coburg.de,/schule/1037.html,50.281584,10.922890
457,1018,Freie Waldorfschule,Freie Waldorfschule Würzburg Schule nach der P...,Oberer Neubergweg 14,97074,Würzburg,www.waldorfschule-wuerzburg.de,/schule/1018.html,49.775040,9.951062


In [1]:
# Write to file to preserve GPS coordinates
schools_of_interest.to_parquet('schools_gps.parquet', engine='pyarrow')

NameError: name 'schools_of_interest' is not defined